This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [119]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [120]:
# First, import the relevant modules
import requests
import collections

In [121]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json'
prms = {'api_key':API_KEY, 'start_date':'2020-04-20', 'end_date':'2020-04-22'}
r = requests.get(url, params = prms)

qnd = r.json()


In [122]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

#The API returns a dict with a single key 'dataset'. The value for that key is a dictionary holding all the information
#for the stock. The 'column_names' key has the name of all the price indicators. The actual numbers are in the 'data' key.
#The values for the 'data' key are stored in a list of lists. Each element of the inner lists corresponds to a column of the
#'column_names' key. Each list represents the price movements of one day. It doesn't return anything for 'Change'
#print(qnd.values())

print(qnd['dataset']['column_names'])
print(qnd['dataset']['data'][1])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
['2020-04-21', 93.4, 93.95, 91.5, 91.5, None, 115152.0, 10603440.3, None, None, None]


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [123]:
#First we need to get a response from the quandl API. Our url is already defined above. We just need to adjust the 
#parameters in order to get the data for 2017
prms = {'api_key':API_KEY, 'start_date':'2017-01-01', 'end_date':'2017-12-31'}

#Using requests.get() in order to send the request and catch the response
res = requests.get(url, params = prms)

In [124]:
#Now we'll use the .json() on the response object in order to get the response as a json object 
#which python will read as a dict
quandl = res.json()

print(quandl.keys())
print(len(quandl['dataset']['data']))
print(quandl['dataset']['column_names'])

dict_keys(['dataset'])
255
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [125]:
#Now we're moving on to finding what the highest and the lowest opening prices were for 2017
# The open price is the second element on every list representing the price movements for each day
#I'll extract all the opening prices along with the dates and then figure out on which days AFX_X had its highest and 
#lowest opening price 

daily = quandl['dataset']['data']

#It'll actually be more convinient to move every indicator to a dict where the keys are the names of the indicators and the
#values are the daily numbers throughout 2017
daily_dict= collections.defaultdict(list)

for day in daily:
    if day[1] != None:
        daily_dict['date'].append(day[0])
        daily_dict['open'].append(day[1])
        daily_dict['high'].append(day[2])
        daily_dict['low'].append(day[3])
        daily_dict['close'].append(day[4])
        daily_dict['volume'].append(day[6])

#Using max() and min() in order to get the highest and lowst open and then match them to their corresponding dates
max_open = max(daily_dict['open'])
max_date = daily_dict['date'][daily_dict['open'].index(max_open)]

min_open = min(daily_dict['open'])
min_date = daily_dict['date'][daily_dict['open'].index(min_open)]

print('The highest opening price was {} on {}'.format(max_open, max_date))
print('The lowest opening price was {} on {}'.format(min_open, min_date))

The highest opening price was 53.11 on 2017-12-14
The lowest opening price was 34.0 on 2017-01-24


In [126]:
#The next step is figuring out what was the biggest intraday movement by finding the biggest difference 
#between high and low
#Using zip() on date, high and low in order to iterate through all of them together. Saving the difference between 
#high and low as a value associated to the corresponding date

hi_low = collections.defaultdict(float)
for date, high, low in zip(daily_dict['date'], daily_dict['high'], daily_dict['low']):
    hi_low[date] = (high - low)

max_diff = round(max(hi_low.values()),2)

#Matching the date to the difference
for date, diff in hi_low.items():
    if diff == max_diff:
        diff_date = date
print('The largest change in one day was {} on {}'.format(max_diff, diff_date))

The largest change in one day was 2.81 on 2017-05-11


In [127]:
#Largest price change between two days based on their closing values.
#Running a loop of the closing values to see what was the biggest change between two days

daily_diff = 0
for i in range(0,len(daily_dict['close'])-1):
    c = abs(daily_dict['close'][i]-daily_dict['close'][i+1])
    if c > daily_diff:
        daily_diff = round(c, 2)

print('The largest change between any two days in 2017 was {}'.format(daily_diff))

The largest change between any two days in 2017 was 2.56


In [128]:
#Getting the average trading volume without libraries

total_vol = sum(daily_dict['volume'])
n_days = len(daily_dict['volume'])

avg_vol = round(total_vol/n_days,2 )

print('The average trading volume during 2017 was {}'.format(avg_vol))

The average trading volume during 2017 was 89140.98


In [129]:
#Getting the median trading volume without libraries

sort_vol = sorted(daily_dict['volume'])
if n_days % 2 == 0: 
    median1 = sort_vol[n_days//2] 
    median2 = sort_vol[n_days//2 - 1] 
    median_vol = (median1 + median2)/2
else: 
    median_vol = sort_vol[n_days//2] 
    
print('The median trading volume for 2017 was {}'.format(median_vol))

The median trading volume for 2017 was 74723.5
